### Data tunning

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
datos = pd.read_csv('/content/2_MOCK_DATA.csv')
datos = datos.sort_values(by=['hora']).reset_index(drop=True)

In [3]:
# Creo la columna 'consumo_del_dia' con datos coherentes
consumo = []
for i in range(1000):
  consumo.append(i/random.randrange(30,38) + i*0.01)
consumo.sort()
consumo = pd.DataFrame(consumo)
consumo = round(consumo,4)
datos = pd.concat([datos, consumo], axis=1)
datos.rename(columns = {0:'consumo_del_dia'}, inplace = True)

In [4]:
# Adecúo el campo 'hora' para que quede cada 30 segundos (como dice la letra)
datos['fecha2'] = '2022-07-20'
datos['hora'] = pd.date_range("12:00:00", "20:19:30", freq="30S").strftime('%H:%M:%S')
datos['fecha_hora'] = datos["fecha2"] + ' ' + datos["hora"]
datos = datos.loc[:, datos.columns != 'fecha']
datos = datos.loc[:, datos.columns != 'fecha2']
datos = datos.loc[:, datos.columns != 'hora']

In [5]:
# Creo la columna 'bateria_restante' con los datos de bateria1 y bateria2
bateria_1 = datos['bateria1']
bateria_1 = pd.DataFrame(bateria_1).sort_values(by='bateria1', ascending=False).reset_index(drop=True).tail(500)
bateria_1 = list(bateria_1['bateria1'])
bateria_2 = datos['bateria2']
bateria_2 = pd.DataFrame(bateria_2).sort_values(by='bateria2', ascending=False).reset_index(drop=True).head(500)
bateria_2 = list(bateria_2['bateria2'])
bateria_restante = bateria_1 + bateria_2
bateria_restante = pd.DataFrame(bateria_restante).reset_index(drop=True)
datos['bateria'] = bateria_restante
datos = datos.loc[:, datos.columns != 'bateria1']
datos = datos.loc[:, datos.columns != 'bateria2']

In [6]:
# Creo la columna 'dist_supercharger' con los datos de dist_supercharger1 y dist_supercharger2
dist_1 = pd.DataFrame(datos['dist_supercharger1'])
dist_1 = dist_1.sort_values(by='dist_supercharger1', ascending=False)
dist_1 = dist_1.tail(500).reset_index(drop=True)
dist_1 = list(dist_1['dist_supercharger1'])
dist_2 = pd.DataFrame(datos['dist_supercharger2'])
dist_2 = dist_2.sort_values(by='dist_supercharger2', ascending=False)
dist_2 = dist_2.head(500).reset_index(drop=True)
dist_2 = list(dist_2['dist_supercharger2'])
dist_supercharger = dist_1 + dist_2
datos['dist_supercharger'] = dist_supercharger
datos = datos.loc[:, datos.columns != 'dist_supercharger1']
datos = datos.loc[:, datos.columns != 'dist_supercharger2']

In [7]:
# Reordeno las columnas
datos = datos[['id_vehiculo', 'fecha_hora', 'ondas', 'temp_motor', 'vibracion', 'presion',
               'humedad','voltaje', 'velocidad','consumo_del_dia','bateria', 'dist_supercharger']]

In [ ]:
consumo = []
for i in range(500,1000):
  consumo.append(datos.at[i,'consumo_del_dia'])

# Como el auto esta apagado al cargarse, los sensores no registran información
for i in range(500,570):
  datos.at[i,'ondas']=np.nan
  datos.at[i,'temp_motor']=np.nan
  datos.at[i,'vibracion']=np.nan
  datos.at[i,'presion']=np.nan
  datos.at[i,'humedad']=np.nan
  datos.at[i,'voltaje']=np.nan
  datos.at[i,'velocidad']=np.nan
  datos.at[i,'bateria']=np.nan
  datos.at[i,'consumo_del_dia']=np.nan
  datos.at[i,'dist_supercharger']=np.nan

# Para que al arrancar el auto, consumo_del_dia empiece desde el ultimo valor
for i,z in enumerate(range(570,1000)):
  datos.at[z, 'consumo_del_dia'] = consumo[i]

datos          

### CSV

In [ ]:
datos.to_csv('/content/data_to_elastic.csv')

### JSON

In [9]:
datos.to_json("./4_data_to_elastic.json", orient="records", lines=True)